In [1]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing


def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

model = EfficientDet()
print_model_summary(model)
load_weights_from_epoch = Config.load_weights_from_epoch
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))

Model: "efficient_det"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net (EfficientNet) multiple                  6771296   
_________________________________________________________________
bi_fpn (BiFPN)               multiple                  129126    
_________________________________________________________________
box_class_predict (BoxClassP multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [2]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')


# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512,512))
    # image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
# Model has only one input so each data point has one element
    yield [image]
    

#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

In [4]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.float32'>


In [5]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

name : input_1
index : 0
shape : [  1 512 512   3]
shape_signature : [  1 512 512   3]
dtype : <class 'numpy.uint8'>
quantization : (1.0, 0)
quantization_parameters : {'scales': array([1.], dtype=float32), 'zero_points': array([0]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential/mb_conv/se_block/global_average_pooling2d/Mean/reduction_indices
index : 1
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module/up_sampling2d/mul
index : 2
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_modul

       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_3/mb_conv_7/se_block_7/conv2d_31/BiasAdd;efficient_det/efficient_net/sequential_4/mb_conv_8/se_block_8/conv2d_35/Conv2D;efficient_det/efficient_net/sequential_3/mb_conv_7/se_block_7/conv2d_31/Conv2D;efficient_det/efficient_net/sequential_3/mb_conv_7/se_block_7/conv2d_31/BiasAdd/ReadVariableOp/resource
index : 91
shape : [480]
shape_signature : [480]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([2.0260133e-08, 2.0134005e-08, 2.0311168e-08, 2.0271445e-08,
       2.0381828e-08, 2.0159460e-08, 2.0341293e-08, 2.0033820e-08,
       2.0383755e-08, 2.0246038e-08, 2.0278186e-08, 2.0028601e-08,
       2.0010049e-08, 1.9659536e-08, 2.0387910e-08, 2.0431960e-08,
       2.0289917e-08, 2.0075488e-08, 2.0118588e-08, 2.0275969e-08,
       2.0124046e-08, 2.0199229e-08, 2.0161782e-08, 2.0392349

       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_4/mb_conv_9/se_block_9/conv2d_39/BiasAdd;efficient_det/efficient_net/sequential_5/mb_conv_11/se_block_11/conv2d_47/Conv2D;efficient_det/efficient_net/sequential_4/mb_conv_9/se_block_9/conv2d_39/Conv2D;efficient_det/efficient_net/sequential_4/mb_conv_9/se_block_9/conv2d_39/BiasAdd/ReadVariableOp/resource
index : 115
shape : [672]
shape_signature : [672]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([1.39180623e-08, 1.40247822e-08, 1.39846943e-08, 1.41823575e-08,
       1.41372052e-08, 1.40524321e-08, 1.39675489e-08, 1.40719409e-08,
       1.40539429e-08, 1.39963499e-08, 1.41392702e-08, 1.40696148e-08,
       1.40080196e-08, 1.39027119e-08, 1.40736418e-08, 1.41280667e-08,
       1.41260887e-08, 1.39623602e-08, 1.41011656e-08, 1.40108485e-08,
       1.40099843e-08, 1.40327474e-08, 1.42003573e-08, 1

       6.6023724e-09, 6.2618066e-09], dtype=float32), 'zero_points': array([0, 0, 0, ..., 0, 0, 0]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_5/mb_conv_13/se_block_13/conv2d_54/Conv2D
index : 156
shape : [ 288    1    1 1152]
shape_signature : [ 288    1    1 1152]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00051036, 0.00050948, 0.00050909, 0.00051219, 0.00050868,
       0.00051202, 0.00051087, 0.00050825, 0.00050997, 0.00050982,
       0.00050777, 0.00050932, 0.00050787, 0.00051206, 0.00051115,
       0.00050977, 0.0005078 , 0.0005107 , 0.00051074, 0.00050819,
       0.00050865, 0.00051042, 0.00051004, 0.0005125 , 0.0005094 ,
       0.00050768, 0.00050859, 0.00051042, 0.00050895, 0.00051122,
       0.00050866, 0.00050921, 0.0005093 , 0.00050923, 0.00050949,
       0.00050995, 0.00051273, 0.0005105 , 0.00050862, 0.00050918,
       0.00051046, 0.00050942, 0.00051179, 0.000510

       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'quantized_dimension': 3}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module/weighted_feature_fusion_5/separable_conv_norm_act_5/separable_conv2d_5/separable_conv2d
index : 231
shape : [64  1  1 64]
shape_signature : [64  1  1 64]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00184705, 0.00183726, 0.00175895, 0.00184061, 0.00184665,
       0.00182089, 0.00181147, 0.0018475 , 0.0018304 , 0.00185457,
       0.0018478 , 0.00182869, 0.00181303, 0.00185678, 0.00182485,
       0.0018446 , 0.00183691, 0.00177712, 0.00181149, 0.00180285,
       0.00176779, 0.00184823, 0.00185288, 0.0018513 , 0.00176539,
       0.00182571, 0.00183649, 0.00184309, 0.00184292, 0.00183441,
       0.00183729, 0.00183276, 0.00185104, 0.00183935, 0.0018213 ,
       0.00184567, 0.00185258, 0.00180486, 0.00185348, 0.00181421,
       0.00180766, 0.00185243, 0.00182073, 0.00184582, 0.

name : efficient_det/bi_fpn/bi_fpn_module_1/weighted_feature_fusion_14/separable_conv_norm_act_14/batch_normalization_68/FusedBatchNormV3;efficient_det/bi_fpn/bi_fpn_module_1/weighted_feature_fusion_14/separable_conv_norm_act_14/separable_conv2d_14/BiasAdd/ReadVariableOp/resource;efficient_det/bi_fpn/bi_fpn_module_1/weighted_feature_fusion_14/separable_conv_norm_act_14/separable_conv2d_14/BiasAdd
index : 325
shape : [64]
shape_signature : [64]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([1.8824453e-08, 1.9604485e-08, 1.9416671e-08, 1.8861542e-08,
       1.9501186e-08, 1.9443222e-08, 1.9545373e-08, 1.9000034e-08,
       1.9251507e-08, 1.9641751e-08, 1.9542947e-08, 1.9358808e-08,
       1.9292870e-08, 1.9736275e-08, 1.9237865e-08, 1.8876126e-08,
       1.9700451e-08, 1.9389697e-08, 1.9579177e-08, 1.9029008e-08,
       1.9815523e-08, 1.9594392e-08, 1.9254818e-08, 1.9565750e-08,
       1.9577650e-08, 1.9673678e-08, 1.9690338e-08, 1.94941

quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_4/mb_conv_8/se_block_8/mul_1
index : 470
shape : [  1  16  16 480]
shape_signature : [  1  16  16 480]
dtype : <class 'numpy.int8'>
quantization : (2.053698517556768e-05, 0)
quantization_parameters : {'scales': array([2.0536985e-05], dtype=float32), 'zero_points': array([0]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_4/mb_conv_8/Sigmoid_1
index : 471
shape : [  1  16  16 480]
shape_signature : [  1  16  16 480]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/efficient_net/sequential_4/mb_conv_8/mul_1
index : 472
shape : [  1  16  16 480]
shape_si

quantization_parameters : {'scales': array([1.4370123e-05], dtype=float32), 'zero_points': array([-12]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/bi_fpn/bi_fpn_module/weighted_feature_fusion_1/separable_conv_norm_act_1/batch_normalization_55/FusedBatchNormV3;efficient_det/bi_fpn/bi_fpn_module/weighted_feature_fusion_1/separable_conv_norm_act_1/separable_conv2d_1/BiasAdd/ReadVariableOp/resource;efficient_det/bi_fpn/bi_fpn_module/weighted_feature_fusion_1/separable_conv_norm_act_1/separable_conv2d_1/BiasAdd;efficient_det/box_class_predict/conv_norm_act_10/conv2d_75/Conv2D_4;efficient_det/bi_fpn/bi_fpn_module/weighted_feature_fusion_1/separable_conv_norm_act_1/separable_conv2d_1/separable_conv2d
index : 603
shape : [ 1 16 16 64]
shape_signature : [ 1 16 16 64]
dtype : <class 'numpy.int8'>
quantization : (7.410763646475971e-05, -3)
quantization_parameters : {'scales': array([7.410764e-05], dtype=float32), 'zero_points': array([-3]), 'quantized_dimension': 0}

sparsity_parameters : {}

name : efficient_det/box_class_predict/conv_norm_act_8/Sigmoid
index : 714
shape : [ 1 64 64 64]
shape_signature : [ 1 64 64 64]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/box_class_predict/conv_norm_act_8/mul
index : 715
shape : [ 1 64 64 64]
shape_signature : [ 1 64 64 64]
dtype : <class 'numpy.int8'>
quantization : (0.0016361913876608014, -37)
quantization_parameters : {'scales': array([0.00163619], dtype=float32), 'zero_points': array([-37]), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det/box_class_predict/conv_norm_act_10/batch_normalization_75/FusedBatchNormV3;efficient_det/box_class_predict/conv_norm_act_10/batch_normalization_75/FusedBatchNormV3_4;efficient_det/box_class_predict/conv_norm_act_10/conv2d_75/BiasAdd/ReadVariableOp/resour

In [6]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [7]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

{'name': 'input_1', 'index': 0, 'shape': array([  1, 512, 512,   3]), 'shape_signature': array([  1, 512, 512,   3]), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.0, 0), 'quantization_parameters': {'scales': array([1.], dtype=float32), 'zero_points': array([0]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'Identity', 'index': 856, 'shape': array([    1, 49104,     5]), 'shape_signature': array([    1, 49104,     5]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [8]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_data = np.array(data, dtype = np.uint8)
    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output= interpreter.get_tensor(output_details['index'])
    return output

In [9]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')

image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

(1, 512, 512, 3)


In [10]:
import numpy as np

model = load_model(model_path)
output = prediction(model, image)

In [12]:
print(output.shape)
print(output)

(1, 49104, 5)
[[[ 0.00297724 -0.03572687  0.         -0.00297724  0.49124447]
  [-0.01786344 -0.0833627   0.02084067 -0.00893172  0.5091079 ]
  [ 0.         -0.04168135 -0.07443098  0.01786344  0.48826724]
  ...
  [-0.00595448  0.03274963  0.05656755  0.03572687  0.49124447]
  [-0.03274963  0.04168135 -0.05061307  0.00893172  0.5388803 ]
  [-0.04763583 -0.04763583 -0.00595448  0.00297724  0.5001762 ]]]
